# **Avance 6**

## *Análisis fundamental*

Este analisis incluye los siguientes componentes:
* Análisis Top-Down (análisis macroeconomico)
* Análisis Bottom-Up (cálculo de indicadores como PER, Capitalización bursátil, Q de Tobin, Valor Intrínseco, Yield anual, ROA, ROE)
* Benchmark
* Calculo múltiplos 
* Valor por descuento de dividendos

### **Análisis Top-Down**

Se realiza un análisis de los factores que pueden afectar el valor de la compañia, estos se pueden encontrar en noticias o estados financieros y pueden ser a nivel global, del sector o industria o cía.

### **Análisis bottom-up**

En este análisis se utilizan indicadores fundamentales y valoración de acciones para comprender el estado de la acción

In [10]:
!pip install yfinance

"pip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [9]:
import pandas as pd
import yfinance as yf
from datetime import datetime

indice_accion = yf.Ticker("MSFT")

inicio = datetime(2021, 1, 1)
fin = datetime(2022, 10, 31)
hist = indice_accion.history(start = inicio, end = fin)
hist

ModuleNotFoundError: No module named 'yfinance'

#### Método PER

Relación precio_accion/utilidad_upa